In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import jqdatasdk as jq

In [2]:
'''
使用JQData调取金融数据
'''
jq.auth('18602198175','qoFbr3rfmwid')


auth success 


In [55]:
count = jq.get_query_count()
print('当日可调用数据总条数为: ', count['total'])
print('当日剩余调用数据条数为: ', count['spare'])

当日可调用数据总条数为:  1000000
当日剩余调用数据条数为:  981897


In [9]:
# 获取平安银行股票数据， XSHE 深圳交易所 XSHG 上海交易所
jq.get_price('000001.XSHE', start_date='2022-09-18', end_date='2023-08-25')

,open,close,high,low,volume,money
2022-09-19,12.24,12.27,12.37,12.18,64752521.0,7.934913e+08
2022-09-20,12.31,12.05,12.33,12.03,91167667.0,1.102212e+09
2022-09-21,12.02,12.13,12.18,11.91,70087061.0,8.454772e+08
2022-09-22,12.04,12.00,12.07,11.96,60041687.0,7.200582e+08
2022-09-23,11.95,12.00,12.13,11.94,60073205.0,7.227031e+08
...,...,...,...,...,...,...
2023-08-21,11.56,11.32,11.58,11.32,70149785.0,7.996414e+08
2023-08-22,11.38,11.37,11.45,11.31,60680809.0,6.899912e+08
2023-08-23,11.35,11.25,11.42,11.24,69180998.0,7.831028e+08
2023-08-24,11.29,11.13,11.32,11.05,129127070.0,1.439197e+09


In [10]:
# valuation是内置市值数据对象（这里表示查询valuation下所有的属性）
q = jq.query(jq.valuation).filter(jq.valuation.code=='000001.XSHE')
df = jq.get_fundamentals(q, '2022-09-18')
df

,id,code,pe_ratio,turnover_ratio,pb_ratio,ps_ratio,pcf_ratio,capitalization,market_cap,circulating_cap,circulating_market_cap,day,pe_ratio_lyr
0,3431565,000001.XSHE,5.968,NaN,0.7124,1.3792,-2.6959,1.940592e+06,2437.3833,1940554.695,2437.3367,2022-09-18,6.7079


In [11]:
# pd.set_option('display.max_rows', None)
# 汽车制造C36， 零售业F52, 其他金融业：J69, 货币金融服务J66
# 计算机、通信和其他电子设备制造业C39
temp = jq.get_industries(date='2023-08-25')
temp

# 筛选出和计算机相关的产业
temp[temp['name'].str.contains('计算机')]

,name,start_date
C39,计算机、通信和其他电子设备制造业,1990-12-19


In [41]:
'''
智能研投
1. 筛选2023年市盈率小于行业平均水平
2. 市值大于行业平均水平的计算机行业股票
3. 然后观察2022年的市盈率表现
'''

# 设置benchmark_date
benchmark_date = '2023-08-25'
# 设置股票范围， 000018.XSHG 上海180金融股，XSHG 上海证券交易所
pool = jq.get_index_stocks('000018.XSHG', date=benchmark_date)

'''
pool1 = jq.get_industry_stocks('J69', date=benchmark_date)
pool2 = jq.get_industry_stocks('J66', date=benchmark_date)

# 得到所有金融业的股票
pool = pool1 + pool2
'''

# 获取计算机行业版块股票
pool = jq.get_industry_stocks('C39', date=benchmark_date)
pool


['000016.XSHE',
 '000020.XSHE',
 '000021.XSHE',
 '000045.XSHE',
 '000050.XSHE',
 '000063.XSHE',
 '000066.XSHE',
 '000100.XSHE',
 '000413.XSHE',
 '000536.XSHE',
 '000547.XSHE',
 '000561.XSHE',
 '000586.XSHE',
 '000636.XSHE',
 '000670.XSHE',
 '000687.XSHE',
 '000725.XSHE',
 '000727.XSHE',
 '000733.XSHE',
 '000748.XSHE',
 '000801.XSHE',
 '000810.XSHE',
 '000823.XSHE',
 '000938.XSHE',
 '000970.XSHE',
 '000977.XSHE',
 '000988.XSHE',
 '001229.XSHE',
 '001270.XSHE',
 '001308.XSHE',
 '001309.XSHE',
 '001314.XSHE',
 '001339.XSHE',
 '001373.XSHE',
 '002017.XSHE',
 '002025.XSHE',
 '002036.XSHE',
 '002045.XSHE',
 '002049.XSHE',
 '002052.XSHE',
 '002055.XSHE',
 '002077.XSHE',
 '002079.XSHE',
 '002089.XSHE',
 '002104.XSHE',
 '002106.XSHE',
 '002119.XSHE',
 '002130.XSHE',
 '002134.XSHE',
 '002137.XSHE',
 '002138.XSHE',
 '002151.XSHE',
 '002156.XSHE',
 '002161.XSHE',
 '002179.XSHE',
 '002180.XSHE',
 '002185.XSHE',
 '002189.XSHE',
 '002194.XSHE',
 '002199.XSHE',
 '002214.XSHE',
 '002217.XSHE',
 '002222

In [42]:
len(pool)

601

In [45]:
q = jq.query(jq.valuation.code, jq.valuation.pe_ratio, jq.valuation.market_cap).filter(jq.valuation.pe_ratio < 15, jq.valuation.pe_ratio > 0, jq.valuation.market_cap > 40, jq.valuation.code.in_(pool))
df = jq.get_fundamentals(q, date=benchmark_date)
df

,code,pe_ratio,market_cap
0,000823.XSHE,14.8427,46.0200
1,001308.XSHE,11.8810,162.8408
2,002130.XSHE,13.6578,84.1612
3,002384.XSHE,12.3391,305.3823
4,002938.XSHE,10.6339,467.5682
5,002955.XSHE,12.0353,45.4529
6,300390.XSHE,3.7676,224.1249
7,301150.XSHE,14.6815,51.6378
8,301358.XSHE,11.4741,299.7965
9,600060.XSHG,13.5846,271.8609


In [50]:
'''
查询股票代码code， 市盈率pe_ratio, 市值market_cap
1. 市盈率>0
2. 在股票范围内（计算机行业内）
3. 按照PE从小到大排序
'''

q = jq.query(jq.valuation.code, jq.valuation.pe_ratio, jq.valuation.market_cap).filter(jq.valuation.pe_ratio > 0, jq.valuation.code.in_(pool)).order_by(jq.valuation.pe_ratio.asc())
df = jq.get_fundamentals(q, date=benchmark_date)

# 计算平均PE，MC
pe_mean = float(df['pe_ratio'].mean())
mc_mean = float(df['market_cap'].mean())
print('满足条件的股票: {}'.format(len(df)))
print('平均PE: {}, 平均MC: {}'.format(pe_mean, mc_mean))

df

满足条件的股票: 450
平均PE: 102.55968466666666, 平均MC: 156.16871555555554


,code,pe_ratio,market_cap
0,300390.XSHE,3.7676,224.1249
1,600888.XSHG,6.5536,103.6819
2,600525.XSHG,9.6812,62.0645
3,002938.XSHE,10.6339,467.5682
4,601231.XSHG,10.9862,301.3071
...,...,...,...
445,688371.XSHG,794.9042,56.5942
446,001309.XSHE,1735.4290,77.0018
447,688079.XSHG,1791.4603,36.2404
448,300241.XSHE,1930.9841,30.8958


In [52]:
'''
筛选出来我们想要的股票
1. 小于PE水平，且PE>0
2. 大于MC平均
3. 在股票范围（计算机行业）
查询股票代码code, PE
按照PE 从小到大排序
'''

q = jq.query(jq.valuation.code, jq.valuation.pe_ratio).filter(jq.valuation.pe_ratio<pe_mean, jq.valuation.market_cap>mc_mean, jq.valuation.pe_ratio>0, jq.valuation.code.in_(pool)).order_by(jq.valuation.pe_ratio.asc())

# 对于筛选出来的股票，查询benchmark_date的数据
df = jq.get_fundamentals(q, benchmark_date)
df

,code,pe_ratio
0,300390.XSHE,3.7676
1,002938.XSHE,10.6339
2,601231.XSHG,10.9862
3,301358.XSHE,11.4741
4,001308.XSHE,11.8810
...,...,...
79,300782.XSHE,80.2537
80,002156.XSHE,81.0047
81,600460.XSHG,82.1751
82,688249.XSHG,94.2471


In [53]:
# 输出筛选出来的84支股票
print(df['code'].values)

['300390.XSHE' '002938.XSHE' '601231.XSHG' '301358.XSHE' '001308.XSHE'
 '002384.XSHE' '600060.XSHG' '300433.XSHE' '002429.XSHE' '002600.XSHE'
 '603296.XSHG' '603228.XSHG' '000733.XSHE' '000063.XSHE' '600271.XSHG'
 '002841.XSHE' '300373.XSHE' '601869.XSHG' '601138.XSHG' '300628.XSHE'
 '600584.XSHG' '688347.XSHG' '002236.XSHE' '002475.XSHE' '002916.XSHE'
 '300136.XSHE' '300866.XSHE' '600563.XSHG' '002180.XSHE' '603712.XSHG'
 '002415.XSHE' '300476.XSHE' '002049.XSHE' '600764.XSHG' '000977.XSHE'
 '002463.XSHE' '600183.XSHG' '002179.XSHE' '688396.XSHG' '000938.XSHE'
 '000988.XSHE' '600562.XSHG' '002465.XSHE' '603019.XSHG' '600839.XSHG'
 '688981.XSHG' '600745.XSHG' '002281.XSHE' '603290.XSHG' '603986.XSHG'
 '002025.XSHE' '688385.XSHG' '300627.XSHE' '000725.XSHE' '300502.XSHE'
 '300408.XSHE' '688036.XSHG' '688002.XSHG' '600498.XSHG' '688146.XSHG'
 '000021.XSHE' '688432.XSHG' '002138.XSHE' '300223.XSHE' '688172.XSHG'
 '002241.XSHE' '689009.XSHG' '688439.XSHG' '300661.XSHE' '301165.XSHE'
 '6884

In [54]:
# 从2023-1-1 到2023-08-25 这些股票的数据
start_date = datetime(2023,1,1)
end_date = datetime(2023,8,25)

# 获取制定的交易日范围
all_trade_days = jq.get_trade_days(start_date=start_date, end_date=end_date)

for i in all_trade_days:
    # 设置第i天的PE数据
    df[i] = jq.get_fundamentals(q, i)['pe_ratio']

df

/var/folders/h4/7cr1cmpn7v5b3x20_9wz8m740000gn/T/ipykernel_20952/3389603094.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = jq.get_fundamentals(q, i)['pe_ratio']
/var/folders/h4/7cr1cmpn7v5b3x20_9wz8m740000gn/T/ipykernel_20952/3389603094.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[i] = jq.get_fundamentals(q, i)['pe_ratio']
/var/folders/h4/7cr1cmpn7v5b3x20_9wz8m740000gn/T/ipykernel_20952/3389603094.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

,code,pe_ratio,2023-01-03,2023-01-04,2023-01-05,2023-01-06,2023-01-09,2023-01-10,2023-01-11,2023-01-12,...,2023-08-14,2023-08-15,2023-08-16,2023-08-17,2023-08-18,2023-08-21,2023-08-22,2023-08-23,2023-08-24,2023-08-25
0,300390.XSHE,3.7676,6.7125,6.5940,6.7554,7.0318,7.0840,7.0097,6.9644,7.0655,...,4.2604,4.1815,4.1534,4.1210,4.0013,3.9379,3.9112,3.7915,3.8324,3.7676
1,002938.XSHE,10.6339,8.8895,8.9277,9.0614,8.9564,9.0805,8.9755,8.9373,8.8991,...,11.8072,11.6221,11.4888,11.4571,11.2830,11.0191,11.1669,10.8397,10.8450,10.6339
2,601231.XSHG,10.9862,11.3892,11.4054,11.5104,11.5993,11.5831,12.1324,12.1038,12.2020,...,11.9532,11.6735,11.5416,11.6623,11.5416,11.3404,11.4530,11.2679,11.4289,10.9862
3,301358.XSHE,11.4741,11.9775,12.0555,12.2476,12.1801,12.3809,12.3915,12.1647,12.3396,...,12.3376,12.6701,12.3872,12.4467,12.2184,12.1837,12.3425,12.0993,12.1241,11.4741
4,001308.XSHE,11.8810,12.0205,12.1016,12.2644,12.2441,12.4004,12.4991,12.4307,12.5686,...,13.7545,13.7585,13.6172,13.3823,13.0092,13.0299,12.9885,12.5463,12.6569,11.8810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,300782.XSHE,80.2537,52.4886,52.0976,53.4067,51.7870,51.7974,52.1096,51.6705,52.0778,...,62.6466,61.8725,77.3180,59.8317,62.0719,72.0047,78.0306,78.4323,82.5467,80.2537
80,002156.XSHE,81.0047,52.5722,53.5457,55.5851,51.9572,51.9386,52.3172,52.4543,52.0976,...,63.5500,63.0734,80.0137,60.4309,64.5650,73.6185,80.2944,82.4585,83.7911,81.0047
81,600460.XSHG,82.1751,53.9518,54.5121,56.1413,53.6848,52.7305,52.9169,53.5457,53.6848,...,65.6377,64.3148,82.0436,62.1268,71.3664,75.3118,80.5063,82.4654,84.7672,82.1751
82,688249.XSHG,94.2471,55.2382,54.8321,59.3388,54.0453,53.0200,53.8239,54.3243,53.8436,...,66.1646,65.6557,85.3467,63.1868,72.2499,80.3508,83.9136,82.8627,98.2256,94.2471
